## Visualizing Intermediate features

In [1]:
!pip install jupyterthemes

from jupyterthemes import jtplot
# you can select an alternative theme's plot style by name
# onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
jtplot.style('monokai')
jtplot.figsize(aspect=4)

from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from torchvision import models, transforms
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn

## Load Pretrained model

In [2]:
model = models.resnet18(pretrained=True)
model.cuda()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Define function that saves intermediate feature maps

In [3]:
# Define hook class
class Hook():
## implement this
    def __init__(self, layer):
        self.hook = layer.register_forward_hook(self.hook_fn)
        
    def hook_fn(self, module, inputs, outputs):
        self.outputs = outputs.cpu().detach().numpy()
        
    def close():
        self.hook.remove()

In [25]:
hook_handles = []

#hook 걸어주기
print(list(model.modules()))
for layer in model.modules():
##implement this
    if type(layer) == nn.Conv2d:
        hook_handles.append(Hook(layer))

    

[ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
 

## Load image

In [26]:
from PIL import Image
from torchvision import transforms as T

image = Image.open('Cat.jpg')
transform = T.Compose([T.Resize((224, 224)), T.ToTensor()])
X = transform(image).unsqueeze(dim=0).to("cuda")

out = model(X) #model forward. forward가 되면서 위에서 걸어준 hook에 의해 feature map들이 저장되게 됩니다. 

In [27]:
# print(hook_handles[0].outputs.shape)
outputs_list = []

for hook in hook_handles:
    outputs_list.append(hook.outputs) #각 hook에 저장된 output들을 outputs_list에 저장해줍니다.
    
print(len(outputs_list))
print(outputs_list[0].shape)

20
(1, 64, 112, 112)


## Visualize intermediate feature maps
### Let us change the layer number and see how the feature map changes!

(64, 112, 112)


In [28]:
act = outputs_list[0][0] #outputs_list 0: layer number, 1: batch, 2: channel, 3: w, 4: h
print(act.shape)

for i in range(act.shape[0]):
    if i == 64: # 64개 channel에 대해서만 visualize
        break
    plt.subplot(8, 8, i+1)
    plt.imshow(act[i, :, :], cmap='gray')
    plt.axis('off')
    plt.savefig('./features.png')

## Visualize CNN filters

In [30]:
# counter to keep count of the conv layers
counter = 0 

# append all the conv layers and their respective weights to the list
model_layer = list(model.modules())

model_weights = [] # we will save the conv layer weights in this list

print(len(model_layer))

for i in range(len(model_layer)):
    if type(model_layer[i]) == nn.Conv2d: # 각 layer가 conv2d일 경우
        counter += 1
        model_weights.append(model_layer[i].weight)
     
print(f"Total convolutional layers: {counter}")

68
Total convolutional layers: 20


torch.Size([64, 3, 7, 7])


In [33]:
# visualize the first conv layer filters
plt.figure(figsize=(20, 17))

print(model_weights[0].shape)

for i, filter in enumerate(model_weights[0].cpu()): # 첫번째 conv layer의 weight visualize
    if i == 64:
        break
#     print(filter.shape) # filter has shape [3, 7, 7] 
    plt.subplot(8, 8, i+1) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
    plt.imshow(filter[0, :, :].detach())
    plt.axis('off')
    plt.savefig('./filter.png')
plt.show()